## Lets start with langchain and  OpenAI 

#### In the quickstart we will se how to:

###### - Get setup with langChain , LangSmith And  LangServe 

###### - use the most basic and common components of LangChain: propmpt templaetes model and output parsers.

###### Build a simple application with Langchain

###### - Trace your application with LangSmith 

###### - Serve your application with LangServe

In [ ]:
import os 
from  dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

## for langsmith tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT_NAME"] = "LangchainFramework"


In [4]:
!pip install -U langchain-groq



   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ------

In [8]:
# import necessary libraries and configure the Groq LLM

import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    model="llama-3.3-70b-versatile", 
    temperature=0
)


In [9]:
# Input and get responce from LLM

response = llm.invoke("What is generative AI?")
print(response.content)

Generative AI refers to a type of artificial intelligence that is capable of generating new, original content, such as images, videos, music, text, or even entire datasets. This is in contrast to traditional AI, which is typically focused on analyzing and processing existing data.

Generative AI models use complex algorithms and neural networks to learn patterns and relationships within a dataset, and then use this knowledge to create new, synthetic data that is similar in style and structure to the original data. These models can be trained on a wide range of data types, including images, videos, audio, and text.

Some common applications of generative AI include:

1. **Image and video generation**: Generative AI can be used to create realistic images and videos of objects, scenes, and people. This has applications in fields such as computer vision, robotics, and entertainment.
2. **Text generation**: Generative AI can be used to generate human-like text, such as articles, stories, an

In [ ]:
## Now we see different chatprompt template with langchain 
## ChatPromptTemplate allows us to create more complex prompts with multiple messages and roles.


from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a an expert in AI engineer. provide me answer based on the question"),
        
        ("user", "{question}"),
    ]
)
prompt

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a an expert in AI engineer. provide me answer based on the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [ ]:
## creating the chain with prompt template

## Chain is basically mean a sequence of actions that are performed in order to achieve a specific task.

chain = prompt | llm
response = chain.invoke({"question": "Explain deep copy and shallow copy in Python?"})
print(response.content)


messages=[SystemMessage(content='You are a an expert in AI engineer. provide me answer based on the question', additional_kwargs={}, response_metadata={}), HumanMessage(content="Create a Node.js project to send emails using AWS SES SDK v3.\n\nRequirements:\n1. Use '@aws-sdk/client-ses'.\n2. Create a 'sesClient.js' to configure the SES client (Region: us-east-1).\n3. Create an 'index.js' with a function 'sendEmail(toAddress, subject, body)'.\n4. Include a 'package.json' with the AWS SDK dependency.\n5. Use placeholder credentials (AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY) in a .env file structure.\n6. Add a README with instructions on how to verify email identities in the AWS Console.\n", additional_kwargs={}, response_metadata={})]


In [ ]:
### Now we use different output parsers
#output parser mean we can parse the output of the LLM into a specific format like JSON, list, etc.

from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
chain = prompt | llm | output_parser
response = chain.invoke({"question": "List down the programming languages used in Data Science"})
print(response)

messages=[SystemMessage(content='You are a an expert in AI engineer. provide me answer based on the question', additional_kwargs={}, response_metadata={}), HumanMessage(content="Create a Node.js project to send emails using AWS SES SDK v3.\n\nRequirements:\n1. Use '@aws-sdk/client-ses'.\n2. Create a 'sesClient.js' to configure the SES client (Region: us-east-1).\n3. Create an 'index.js' with a function 'sendEmail(toAddress, subject, body)'.\n4. Include a 'package.json' with the AWS SDK dependency.\n5. Use placeholder credentials (AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY) in a .env file structure.\n6. Add a README with instructions on how to verify email identities in the AWS Console.\n", additional_kwargs={}, response_metadata={})]


In [33]:
import os
import re
import shutil
import subprocess
from dotenv import load_dotenv

from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools import DuckDuckGoSearchRun


class SuperiorDeveloperAgent:

    def __init__(self):
        print("Initializing SuperiorDeveloperAgent (Groq)...")

        load_dotenv()
        os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

        self.llm = ChatGroq(
            model="llama-3.3-70b-versatile",
            temperature=0
        )

        self.parser = StrOutputParser()
        self.search = DuckDuckGoSearchRun()

    def parse_and_create(self, text, base_dir="converted_project"):
        if "FILE:" not in text:
            raise ValueError(" No FILE blocks found in model output")

        if os.path.exists(base_dir):
            shutil.rmtree(base_dir)

        os.makedirs(base_dir, exist_ok=True)

        files = re.findall(
            r'FILE:\s*([\w\.\-/]+)\n(.*?)(?=FILE:|$)',
            text,
            re.DOTALL
        )

        for filename, content in files:
            full_path = os.path.join(base_dir, filename.strip())
            os.makedirs(os.path.dirname(full_path), exist_ok=True)

            with open(full_path, "w", encoding="utf-8") as f:
                f.write(content.strip())

            print(f" Created: {full_path}")

    def run_conversion(self, prompt_text, target_lang="Node.js"):
        full_prompt = f"""
You are a Senior Software Engineer.

Create a COMPLETE {target_lang} project.

STRICT RULES:
- Output ONLY file contents
- EVERY file MUST start with: FILE: path/to/file
- NO markdown
- NO explanation

User request:
{prompt_text}
"""

        print("Generating project using Groq...")
        response = self.llm.invoke(full_prompt)

        self.parse_and_create(response.content)
        self.run_npm_install()

    def run_npm_install(self, base_dir="converted_project"):
        package_json = os.path.join(base_dir, "package.json")
        if not os.path.exists(package_json):
            print(" package.json not found, skipping npm install")
            return

        print("Installing npm dependencies...")

        subprocess.run(
            "npm install",
            cwd=base_dir,
            shell=True   # ✅ Windows-safe
        )


In [34]:
agent = SuperiorDeveloperAgent()


agent.run_conversion("""
Create a COMPLETE Node.js project to send emails using AWS SES SDK v3.

STRICT RULES (DO NOT BREAK):
- Output ONLY file contents
- EVERY file MUST start with: FILE: relative/path/filename
- NO explanations, NO markdown, NO extra text

Requirements:
1. Use '@aws-sdk/client-ses'.
2. Create 'sesClient.js' to configure SES client (Region: us-east-1).
3. Create 'index.js' with a function:
   sendEmail(toAddress, subject, body)
4. Include a 'package.json' with required dependencies.
5. Include a '.env.example' file with:
   AWS_ACCESS_KEY_ID=
   AWS_SECRET_ACCESS_KEY=
6. Add a README.md explaining:
   - How to verify email identities in AWS SES Console
   - How to run the project locally
""", target_lang="Node.js")


Initializing SuperiorDeveloperAgent (Groq)...
Generating project using Groq...
 Created: converted_project\package.json
 Created: converted_project\.env.example
 Created: converted_project\sesClient.js
 Created: converted_project\index.js
 Created: converted_project\README.md
Installing npm dependencies...
